## Reference Image

In [1]:
import cv2
import numpy as np

In [2]:
# Load your images
img1 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical2.png', cv2.IMREAD_GRAYSCALE)
fused_img = cv2.imread('D:/SDP/fused_PCA.png', cv2.IMREAD_GRAYSCALE)

def resize_images(*images):
    # Find the smallest dimensions in the set
    min_height = min(image.shape[0] for image in images)
    min_width = min(image.shape[1] for image in images)
    resized_images = [cv2.resize(img, (min_width, min_height), interpolation=cv2.INTER_AREA) for img in images]
    return resized_images


img1, img2, fused_img = resize_images(img1, img2, fused_img)

In [3]:
import numpy as np

def compute_ncc(image1, image2):
    """
    Compute the Nonlinear Correlation Coefficient (NCC) between two images.
    
    Parameters:
        image1: numpy.ndarray, source image 1
        image2: numpy.ndarray, source image 2
    
    Returns:
        float, NCC value
    """
    mean1 = np.mean(image1)
    mean2 = np.mean(image2)
    std1 = np.std(image1)
    std2 = np.std(image2)
    
    NCC = np.mean((image1 - mean1) * (image2 - mean2)) / (std1 * std2)
    
    return NCC

def compute_ncc_matrix(img1, img2, fused_img):
    """
    Compute the nonlinear correlation matrix R using NCC values.
    
    Parameters:
        img1: numpy.ndarray, source image 1
        img2: numpy.ndarray, source image 2
        fused_img: numpy.ndarray, fused image
    
    Returns:
        numpy.ndarray, nonlinear correlation matrix R
    """
    NCCA_B = compute_ncc(img1, img2)
    NCCA_F = compute_ncc(img1, fused_img)
    NCCB_A = compute_ncc(img2, img1)
    NCCB_F = compute_ncc(img2, fused_img)
    NCCF_A = compute_ncc(fused_img, img1)
    NCCF_B = compute_ncc(fused_img, img2)
    
    R = np.array([[1, NCCA_B, NCCA_F],
                  [NCCB_A, 1, NCCB_F],
                  [NCCF_A, NCCF_B, 1]])
    
    return R

def compute_qncie(img1, img2, fused_img):
    """
    Compute the Nonlinear Correlation Information Entropy (QNCIE) between source images and fused image.
    
    Parameters:
        img1: numpy.ndarray, source image 1
        img2: numpy.ndarray, source image 2
        fused_img: numpy.ndarray, fused image
    
    Returns:
        float, QNCIE value
    """
    R = compute_ncc_matrix(img1, img2, fused_img)
    eigenvalues = np.linalg.eigvals(R)
    qncie = 1 + np.sum((eigenvalues/3) * np.log2(eigenvalues/3)) / np.log2(256)
    
    return qncie

# Example usage:
qncie_value = compute_qncie(img1, img2, fused_img)
print("QNCIE value:", qncie_value)

QNCIE value: 0.9220934482333395


In [4]:
from sklearn.feature_selection import mutual_info_regression

def compute_mi(image1, image2):
    """
    Compute Mutual Information (MI) between two images.
    
    Parameters:
        image1: numpy.ndarray, source image 1
        image2: numpy.ndarray, source image 2
    
    Returns:
        float, MI value
    """
    # Flatten images to 1D arrays
    flat_image1 = image1.flatten()
    flat_image2 = image2.flatten()
    
    # Compute mutual information
    mi = mutual_info_regression(flat_image1.reshape(-1, 1), flat_image2)[0]
    
    return mi

def compute_fmi(img1, img2, fused_img):
    """
    Compute the Feature Mutual Information (FMI) between source images and fused image.
    
    Parameters:
        img1: numpy.ndarray, source image 1
        img2: numpy.ndarray, source image 2
        fused_img: numpy.ndarray, fused image
    
    Returns:
        float, FMI value
    """
    # Compute Mutual Information between each source image and fused image
    mi_af = compute_mi(img1, fused_img)
    mi_bf = compute_mi(img2, fused_img)
    
    # Sum up the Mutual Information values
    fmi = mi_af + mi_bf
    
    return fmi

# Example usage:
fmi_value = compute_fmi(img1, img2, fused_img)
print("FMI value:", fmi_value)


FMI value: 2.690387698448898


In [5]:
import numpy as np

def compute_entropy(image):
    """
    Compute the entropy of an image.
    
    Parameters:
        image: numpy.ndarray, input image
    
    Returns:
        float, entropy value
    """
    # Flatten image to 1D array
    flat_image = image.flatten()
    
    # Compute histogram
    hist, _ = np.histogram(flat_image, bins=256, density=True)
    
    # Remove zeros to avoid log(0) issues
    hist = hist[hist != 0]
    
    # Compute entropy
    entropy = -np.sum(hist * np.log2(hist))
    
    return entropy

def compute_mi(image1, image2):
    """
    Compute Mutual Information (MI) between two images.
    
    Parameters:
        image1: numpy.ndarray, source image 1
        image2: numpy.ndarray, source image 2
    
    Returns:
        float, MI value
    """
    # Flatten images to 1D arrays
    flat_image1 = image1.flatten()
    flat_image2 = image2.flatten()
    
    # Compute joint histogram
    joint_hist, _, _ = np.histogram2d(flat_image1, flat_image2, bins=256, density=True)
    
    # Remove zeros to avoid log(0) issues
    joint_hist = joint_hist[joint_hist != 0]
    
    # Compute mutual information
    mi = -np.sum(joint_hist * np.log2(joint_hist))
    
    return mi

def compute_nmi(img1, img2, fused_img):
    """
    Compute the Normalized Mutual Information (NMI) between source images and fused image.
    
    Parameters:
        img1: numpy.ndarray, source image 1
        img2: numpy.ndarray, source image 2
        fused_img: numpy.ndarray, fused image
    
    Returns:
        float, NMI value
    """
    # Compute entropy of each image
    entropy_a = compute_entropy(img1)
    entropy_b = compute_entropy(img2)
    entropy_f = compute_entropy(fused_img)
    
    # Compute mutual information between each source image and fused image
    mi_af = compute_mi(img1, fused_img)
    mi_bf = compute_mi(img2, fused_img)
    
    # Compute NMI
    nmi = (2 * (mi_af / (entropy_a + entropy_f))) + (2 * (mi_bf / (entropy_b + entropy_f)))
    
    return nmi

# Example usage:
nmi_value = compute_nmi(img1, img2, fused_img)
print("NMI value:", nmi_value)


NMI value: 3.089800446832877


In [6]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

def mutual_information(image1, image2):
    hist_2d, _, _ = np.histogram2d(image1.ravel(), image2.ravel(), bins=256)
    pxy = hist_2d / float(np.sum(hist_2d))
    px = np.sum(pxy, axis=1)
    py = np.sum(pxy, axis=0)
    px_py = px[:, None] * py[None, :]
    nzs = pxy > 0
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

def resize_images(*images):
    # Find the smallest dimensions in the set
    min_height = min(image.shape[0] for image in images)
    min_width = min(image.shape[1] for image in images)
    resized_images = [cv2.resize(img, (min_width, min_height), interpolation=cv2.INTER_AREA) for img in images]
    return resized_images

def calculate_metrics(img1, img2, fused_img):
    metrics = {}
    # Resize images to the smallest common dimensions
    img1, img2, fused_img = resize_images(img1, img2, fused_img)
    metrics['SSIM_CT'] = ssim(img1, fused_img, data_range=fused_img.max() - fused_img.min())
    metrics['SSIM_MRI'] = ssim(img2, fused_img, data_range=fused_img.max() - fused_img.min())
    metrics['MSE_CT'] = mean_squared_error(img1, fused_img)
    metrics['MSE_MRI'] = mean_squared_error(img2, fused_img)
    metrics['MI_CT'] = mutual_information(img1, fused_img)
    metrics['MI_MRI'] = mutual_information(img2, fused_img)
    # Add more metrics here if needed
    return metrics

# Load your images
img1 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical2.png', cv2.IMREAD_GRAYSCALE)
fused_img = cv2.imread('D:/SDP/fused_PCA.png', cv2.IMREAD_GRAYSCALE)

# Calculate the metrics
results = calculate_metrics(img1, img2, fused_img)
print(results)


{'SSIM_CT': 0.38980062162989865, 'SSIM_MRI': 0.6020671255162535, 'MSE_CT': 1151.8423453493547, 'MSE_MRI': 1276.1882510013352, 'MI_CT': 1.4306246921766002, 'MI_MRI': 2.1500639717325467}


## No-Reference

In [7]:
import cv2
import numpy as np

def calculate_entropy(image):
    """Calculate the entropy of an image."""
    # Convert image to grayscale if it's not already
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate histogram from 0 to 255
    hist = cv2.calcHist([image], [0], None, [256], [0,256])
    # Normalize the histogram
    hist_norm = hist.ravel()/hist.sum()
    # Calculate the entropy
    logs = np.log2(hist_norm + 1e-10)  # Add a small value to avoid log(0)
    entropy = -1 * (hist_norm * logs).sum()
    
    return entropy

# Load your fused image
fused_img = cv2.imread('D:/SDP/fused_PCA.png', cv2.IMREAD_GRAYSCALE)

# Calculate the entropy
entropy_value = calculate_entropy(fused_img)
print(f"Entropy: {entropy_value}")



Entropy: 6.000565052032471


In [8]:
import cv2
import numpy as np

def calculate_average_gradient(image):
    """Calculate the average gradient of an image."""
    # Ensure the image is in grayscale
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate gradients in the x and y directions
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Calculate the magnitude of the gradients
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    # Calculate the average gradient
    average_gradient = np.mean(grad_magnitude)
    
    return average_gradient

# Load your fused image
fused_img = cv2.imread('D:/SDP/fused_PCA.png', cv2.IMREAD_GRAYSCALE)

# Calculate the average gradient
average_gradient_value = calculate_average_gradient(fused_img)
print(f"Average Gradient: {average_gradient_value}")


Average Gradient: 77.66337745210586


In [9]:
import cv2
import numpy as np

def edge_similarity(image1, image2, fused_image):
    """Calculate the edge-based similarity measurement between source images and fused image."""
    # Compute gradients for source images and fused image
    grad_x1, grad_y1 = cv2.Sobel(image1, cv2.CV_64F, 1, 0), cv2.Sobel(image1, cv2.CV_64F, 0, 1)
    grad_x2, grad_y2 = cv2.Sobel(image2, cv2.CV_64F, 1, 0), cv2.Sobel(image2, cv2.CV_64F, 0, 1)
    grad_xf, grad_yf = cv2.Sobel(fused_image, cv2.CV_64F, 1, 0), cv2.Sobel(fused_image, cv2.CV_64F, 0, 1)
    
    # Calculate gradient magnitudes and angles
    magnitude1, angle1 = cv2.cartToPolar(grad_x1, grad_y1)
    magnitude2, angle2 = cv2.cartToPolar(grad_x2, grad_y2)
    magnitudef, anglef = cv2.cartToPolar(grad_xf, grad_yf)
    
    # Calculate edge information retention (intensity and orientation)
    qa_f = magnitude1 * (1 - np.abs(np.sin(angle1 - anglef)))
    qb_f = magnitude2 * (1 - np.abs(np.sin(angle2 - anglef)))
    
    # Define weights for each source image (example: equal weights)
    wa = wb = 0.5  # Adjust these weights as per the importance of each source image
    
    # Calculate edge-based similarity measure
    numerator = np.sum(qa_f * wa + qb_f * wb)
    denominator = np.sum(wa + wb)
    
    qab_f = numerator / denominator if denominator != 0 else 0
    
    return qab_f
    
def resize_images(*images):
    # Find the smallest dimensions in the set
    min_height = min(image.shape[0] for image in images)
    min_width = min(image.shape[1] for image in images)
    resized_images = [cv2.resize(img, (min_width, min_height), interpolation=cv2.INTER_AREA) for img in images]
    return resized_images

# Load your images
img1 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical2.png', cv2.IMREAD_GRAYSCALE)
fused_img = cv2.imread('D:/SDP/fused_PCA.png', cv2.IMREAD_GRAYSCALE)


img1, img2, fused_img = resize_images(img1, img2, fused_img)

# Calculate the edge-based similarity measurement
qab_f_value = edge_similarity(img1, img2, fused_img)
print(f"Edge-based Similarity Measurement (QAB/F): {qab_f_value}")


Edge-based Similarity Measurement (QAB/F): 653820.2059047013


In [10]:
import cv2
import numpy as np

def calculate_standard_deviation(image):
    """Calculate the standard deviation of an image."""
    # Ensure the image is in grayscale
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate mean
    mean_value = np.mean(image)
    # Calculate standard deviation
    standard_deviation = np.sqrt(np.mean((image - mean_value) ** 2))
    
    return standard_deviation

# Load your fused image
fused_img = cv2.imread('D:/SDP/fused_PCA.png', cv2.IMREAD_GRAYSCALE)

# Calculate the standard deviation
standard_deviation_value = calculate_standard_deviation(fused_img)
print(f"Standard Deviation: {standard_deviation_value}")


Standard Deviation: 47.137969156076544


In [11]:
import cv2
import numpy as np

def calculate_spatial_frequency(image):
    """Calculate the spatial frequency of an image."""
    # Ensure the image is in grayscale
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate differences between adjacent pixels along rows and columns
    row_diff = np.diff(image.astype('float'), axis=0)
    col_diff = np.diff(image.astype('float'), axis=1)
    
    # Calculate the squares of differences
    RF_squared = np.sum(row_diff ** 2)
    CF_squared = np.sum(col_diff ** 2)
    
    # Calculate row frequency (RF) and column frequency (CF)
    RF = np.sqrt(RF_squared / (image.shape[0] * image.shape[1]))
    CF = np.sqrt(CF_squared / (image.shape[0] * image.shape[1]))
    
    # Calculate spatial frequency (SF)
    SF = np.sqrt(RF ** 2 + CF ** 2)
    
    return SF

# Load your fused image
fused_img = cv2.imread('D:/SDP/fused_PCA.png', cv2.IMREAD_GRAYSCALE)

# Calculate the spatial frequency
spatial_frequency_value = calculate_spatial_frequency(fused_img)
print(f"Spatial Frequency: {spatial_frequency_value}")


Spatial Frequency: 28.738310411589946


In [12]:
import cv2
import numpy as np

def calculate_edge_intensity(image):
    """Calculate the edge intensity of an image using Sobel operators."""
    # Ensure the image is in grayscale
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Sobel operators to find horizontal and vertical gradients
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Calculate the squares of gradients
    S2x = np.sum(sobel_x**2)
    S2y = np.sum(sobel_y**2)
    
    # Calculate Edge Intensity
    EI = np.sqrt(S2x + S2y)
    
    return EI

# Load your fused image
fused_img = cv2.imread('D:/SDP/fused_PCA.png', cv2.IMREAD_GRAYSCALE)

# Calculate the edge intensity
edge_intensity_value = calculate_edge_intensity(fused_img)
print(f"Edge Intensity: {edge_intensity_value}")


Edge Intensity: 14970.639532097484
